In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
size = [224, 224]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
training_data = '/content/drive/MyDrive/HDA2/Covid19-dataset/train'
test_data = '/content/drive/MyDrive/HDA2/Covid19-dataset/test'

In [ ]:
vgg = VGG16(input_shape=size + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg.layers:
  layer.trainable = False

In [ ]:
data = glob('/content/drive/MyDrive/HDA2/Covid19-dataset/train/*')

In [ ]:
x = Flatten()(vgg.output)
prediction = Dense(len(data), activation='softmax')(x)

In [ ]:
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_img = ImageDataGenerator(rescale = 1./255,shear_range = 0.25,zoom_range = 0.3)
test_img = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_img.flow_from_directory('/content/drive/MyDrive/HDA2/Covid19-dataset/train',target_size=size)

Found 251 images belonging to 3 classes.


In [ ]:
testing_set = test_img.flow_from_directory('/content/drive/MyDrive/HDA2/Covid19-dataset/test',
                                            target_size = (224, 224),
                                            batch_size = 8,
                                            class_mode = 'categorical')

Found 66 images belonging to 3 classes.


In [ ]:

r = model.fit(
  training_set,
  validation_data=testing_set,
  epochs=6,
  steps_per_epoch=len(training_set),
  validation_steps=len(testing_set)
)

Epoch 1/6
8/8 [==============================] - 134s 16s/step - loss: 1.1719 - accuracy: 0.5657 - val_loss: 0.9283 - val_accuracy: 0.5909
Epoch 2/6
8/8 [==============================] - 123s 16s/step - loss: 0.4560 - accuracy: 0.7968 - val_loss: 0.2575 - val_accuracy: 0.9242
Epoch 3/6
8/8 [==============================] - 123s 16s/step - loss: 0.2259 - accuracy: 0.9004 - val_loss: 0.2070 - val_accuracy: 0.9242
Epoch 4/6
8/8 [==============================] - 123s 16s/step - loss: 0.1756 - accuracy: 0.9283 - val_loss: 0.2052 - val_accuracy: 0.9091
Epoch 5/6
8/8 [==============================] - 124s 16s/step - loss: 0.1454 - accuracy: 0.9562 - val_loss: 0.1602 - val_accuracy: 0.9697
Epoch 6/6
8/8 [==============================] - 140s 18s/step - loss: 0.1087 - accuracy: 0.9761 - val_loss: 0.1936 - val_accuracy: 0.9091


In [ ]:
model.save('Lung_disease_pred_vgg.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np


model_vgg = load_model('/content/drive/MyDrive/HDA2/Lung_disease_pred_vgg.h5')

In [ ]:

y_pred_vgg = model_vgg.predict(testing_set)


y_pred_classes_vgg = np.argmax(y_pred_vgg, axis=1)
y_true_classes = testing_set.classes

accuracy_vgg = accuracy_score(y_true_classes, y_pred_classes_vgg)
print("Accuracy (vgg):", accuracy_vgg)


print("Classification Report (vgg):")
print(classification_report(y_true_classes, y_pred_classes_vgg))


conf_matrix_vgg = confusion_matrix(y_true_classes, y_pred_classes_vgg)
print("Confusion Matrix (vgg):")
print(conf_matrix_vgg)

9/9 [==============================] - 25s 3s/step
Accuracy (vgg): 0.30303030303030304
Classification Report (vgg):
              precision    recall  f1-score   support

           0       0.31      0.31      0.31        26
           1       0.28      0.35      0.31        20
           2       0.33      0.25      0.29        20

    accuracy                           0.30        66
   macro avg       0.31      0.30      0.30        66
weighted avg       0.31      0.30      0.30        66

Confusion Matrix (vgg):
[[ 8 12  6]
 [ 9  7  4]
 [ 9  6  5]]


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay

auroc_per_class = [roc_auc_score(y_true_classes == i, y_pred_vgg[:, i]) for i in range(len(data))]


for i, auroc in enumerate(auroc_per_class):
    print(f"Class {i} AUROC (vgg): {auroc}")



Class 0 AUROC (vgg): 0.42500000000000004
Class 1 AUROC (vgg): 0.42717391304347824
Class 2 AUROC (vgg): 0.5358695652173913


In [ ]:
from PIL import Image
image = '/content/drive/MyDrive/Covid19-dataset/test/Viral Pneumonia/0101.jpeg'
image1 = Image.open(image)

In [ ]:
new_size = (224,224)


resized_image = image1.resize(new_size)


resized_image.save('output_image.jpg')

In [ ]:
resized_image = np.array(resized_image)

In [ ]:
x = np.expand_dims(resized_image,axis=0)
x = preprocess_input(x)

In [ ]:
a = model.predict(x)

In [ ]:
  new_size = (224,224)


resized_image = image1.resize(new_size)


resized_image.save('output_image.jpg')

In [ ]:
resized_image = np.array(resized_image)
x = np.expand_dims(resized_image,axis=0)
x = preprocess_input(x)

In [ ]:
x.shape

In [ ]:
np.argmax(model.predict(x))